In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# creating classifier

In [ ]:
autoencoder_model = tf.keras.models.load_model(
    r"./saved_models/encoder_v3",
    custom_objects={"custom_loss": None},
)

autoencoder_model.trainable = False
model_input = autoencoder_model.layers[0].input
encoder_output = autoencoder_model.get_layer("encoder_output").output
encoder = keras.Model(inputs=[model_input], outputs=[encoder_output])

x = x_in = keras.layers.Input((256, 256, 3))

encoder_output = encoder(x)


x = keras.layers.Flatten()(encoder_output)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(5, "softmax")(x)

# create model
classifier = keras.Model(inputs=[x_in], outputs=[x])

classifier.summary()

# load classifier

In [ ]:
classifier = tf.keras.models.load_model(r"./saved_models/classifier_v2")

classifier.summary(expand_nested=True)

In [ ]:
for l in classifier.layers:
    if isinstance(l, tf.keras.Model):
        for l2 in l.layers:
            print(l2)

    print(l.name)

In [ ]:
denseLayer = classifier.get_layer(name="dense")

weights, bias = denseLayer.get_weights()

np.save("./data/dense_weights", weights)
np.save("./data/dense_bias", bias)

# create the numpy layer

In [ ]:
from FullyConectedLayer import FullyConnectedLayer
from Softmax import Softmax

softmax = Softmax()
fc = FullyConnectedLayer("dense", activation=softmax)
fc.load_weights(
    "./data/dense_weights.npy",
    "./data/dense_bias.npy",
)

# test 

In [ ]:
# test the tensorflow layer and the numpy layer on the same input

input_test = np.ones((32,4096))

print(fc(input_test, False))
print(denseLayer(input_test).numpy())